In [1]:
import os
from utils import DataSeq, load_ids, load_pickle_ma

import numpy as np
import pandas as pd
import skimage.io as io

from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils import shuffle

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import F1Score
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.losses import Loss

import keras_tuner as kt

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

2023-11-28 13:10:52.350693: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-28 13:10:52.385769: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-28 13:10:52.897844: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
pkl_path = "./train_crowdsourced_labels.pkl"
images_path = "./Train_non_experts_simple/"
test_path = "./Test/"
batch_size = 8
epochs = 10
R = 20
K = 3

In [6]:
ids, labels = load_pickle_ma(pkl_path, images_path, R)
df["annotations"][0]

array([[13,  1],
       [23,  1],
       [17,  2],
       [ 7,  1],
       [ 5,  1],
       [11,  1],
       [12,  1],
       [21,  1],
       [19,  1],
       [ 8,  1],
       [10,  1],
       [22,  1],
       [14,  1],
       [24,  1],
       [20,  1],
       [ 9,  1],
       [15,  1]])

In [ ]:
class GCCELoss(Loss):
    def __init__(self, R=20, K=3, q=):
        super().__init__()
        self.R = R
        self.K = K
        self.q = q
        

    def call(self, y_true, y_pred):
        ann = y_pred[:, :self.R]
        pred = y_pred[:, self.R:]
        pred = tf.clip_by_value(pred, clip_value_min=1e-9, clip_value_max=1-1e-9)
        y_true_onehot = tf.one_hot(tf.cast(y_true, dtype=tf.int32), depth=self.K, axis=1)
        y_hat = tf.repeat(tf.expand_dims(pred,-1), self.R, axis = -1)
        p_gcce = y_true_onehot*(1 - Y_hat**self.q)/self.q
        temp1 = ann*tf.math.reduce_sum(p_gcce, axis=1)
        temp2 = (1 - ann)*(1-(1/self.K)**self.q)/self.q*tf.reduce_sum(Y_true,axis=1)
        return tf.math.reduce_sum((temp1 + temp2))
        

In [ ]:
R = 3
K = 5
y_pred = tf.constant([[0.7, 0.8, 0.4, 0.6, 0.15, 0.1, 0.1, 0.05],
                      [0.6, 0.7, 0.5, 0.5, 0.2, 0.1, 0.1, 0.1],
                      [0.65, 0.82, 0.34, 0.6, 0.15, 0.1, 0.1, 0.05]])



pred = y_pred[:, R:]
ann = y_pred[:, :R]

print("PRED", pred)
print("ANN", ann)

Y_hat = tf.repeat(tf.expand_dims(pred,-1), R, axis = -1)
print(Y_hat)


In [ ]:
y_true = tf.constant([[1, 3, 0], [2, 4, 2], [-1, 4, 2]])
Y_true = tf.one_hot(tf.cast(y_true, dtype=tf.int32), depth=K, axis=1)
print(Y_true)